## Cluster

In [ ]:
DATA_DIR = './miocarditeDataset'
NORMAL_DIR = './miocarditeDataset/Normal'
ANORMAL_DIR = './miocarditeDataset/Abnormal'
SICK_DIR = './miocarditeDataset/Sick/*/*'
SICK_DIR_DS = './miocarditeDataset/Sick'

In [ ]:
from glob import glob
import os
import PIL
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
pathImgNormal = glob(os.path.join(NORMAL_DIR,'*.jpg'))
pathImgAnormal = glob(os.path.join(ANORMAL_DIR,'*.jpg'))
pathImgSick = glob(os.path.join(SICK_DIR,'*.jpg'))
print('Numero de imgs normais: {}'.format(len(pathImgNormal)))
print('Numero de imgs com miocardite: {}'.format(len(pathImgAnormal)))
print('Numero de imgs doentes: {}'.format(len(pathImgSick)))
pathImgData = pathImgNormal + pathImgAnormal + pathImgSick
print('Total de registros: {}'.format(len(pathImgData)))

In [ ]:
!pip install kmeans-pytorch

In [ ]:
## K-Means

from PIL import Image
import torch
import torchvision.transforms as T
from kmeans_pytorch import kmeans, kmeans_predict

In [ ]:
prep_transform = T.Compose([
                  T.ToTensor(),
                  T.Resize(size=(100,100)),
                ])

images_normal = []
images_anormal = []

for img in pathImgNormal:
    image = Image.open(img)
    image = prep_transform(image)
    image = torch.flatten(image,1)
    images_normal.append(image.numpy())

for img in pathImgAnormal:
    image = Image.open(img)
    image = prep_transform(image)
    image = torch.flatten(image,1)
    images_anormal.append(image.numpy())

images_normal = torch.tensor(np.array(images_normal))
images_anormal = torch.tensor(np.array(images_anormal))

print(images_normal.shape)
print(images_anormal.shape)

In [ ]:
data_size, dims, num_clusters = 2449, 10000, 2
data_size2 = 4686

cluster_ids_x, cluster_centers = kmeans(
    X=images_normal, num_clusters=num_clusters, distance='euclidean',device=torch.device('cuda:0')
)

cluster_ids_x2, cluster_centers2 = kmeans(
    X=images_anormal, num_clusters=num_clusters, distance='euclidean',device=torch.device('cuda:0')
)

# cluster IDs and cluster centers normal
print(cluster_ids_x)
print(cluster_centers)

# cluster IDs and cluster centers anormal
print(cluster_ids_x2)
print(cluster_centers2)

print(np.count_nonzero(cluster_ids_x == 0))
print(np.count_nonzero(cluster_ids_x == 1))

print(np.count_nonzero(cluster_ids_x2 == 0))
print(np.count_nonzero(cluster_ids_x2 == 1))

In [ ]:
NORMAL0_DIR = DATA_DIR + "/normal0"
NORMAL1_DIR = DATA_DIR + "/normal1"
ANORMAL2_DIR = DATA_DIR + "/anormal2"
ANORMAL3_DIR = DATA_DIR + "/anormal3"

In [ ]:
# Limpando os diretorios
filelist = glob(os.path.join(NORMAL0_DIR,"*"))
for f in filelist:
    os.remove(f)
filelist = glob(os.path.join(NORMAL1_DIR,"*"))
for f in filelist:
    os.remove(f)
filelist = glob(os.path.join(ANORMAL2_DIR,"*"))
for f in filelist:
    os.remove(f)
filelist = glob(os.path.join(ANORMAL3_DIR,"*"))
for f in filelist:
    os.remove(f)

In [ ]:
import shutil

for i in range(len(cluster_ids_x)):
    if(cluster_ids_x[i] == 0):
        shutil.copy2(pathImgNormal[i],NORMAL0_DIR)
    else:
        shutil.copy2(pathImgNormal[i],NORMAL1_DIR)

print(len(os.listdir(NORMAL0_DIR)))
print(len(os.listdir(NORMAL1_DIR)))

for i in range(len(cluster_ids_x2)):
    if(cluster_ids_x2[i] == 0):
        shutil.copy2(pathImgAnormal[i],ANORMAL2_DIR)
    else:
        shutil.copy2(pathImgAnormal[i],ANORMAL3_DIR)

print(len(os.listdir(ANORMAL2_DIR)))
print(len(os.listdir(ANORMAL3_DIR)))


In [ ]:
## Criando CSV

import pandas as pd

images2 = []
targets2 = []

path_normal0 = glob(os.path.join(NORMAL0_DIR,'*.jpg'))
path_normal1 = glob(os.path.join(NORMAL1_DIR,'*.jpg'))
path_anormal2 = glob(os.path.join(ANORMAL2_DIR,'*.jpg'))
path_anormal3 = glob(os.path.join(ANORMAL3_DIR,'*.jpg'))


for img in path_normal0:
  images2.append(img.split('/')[-1])
  targets2.append(0)

for img in path_normal1:
  images2.append(img.split('/')[-1])
  targets2.append(1)

for img in path_anormal2:
  images2.append(img.split('/')[-1])
  targets2.append(2)

for img in path_anormal3:
  images2.append(img.split('/')[-1])
  targets2.append(3)

for img in pathImgSick:
  aux = img.split('/')[2:]
  images2.append(aux[1] + '/' + aux[2] + '/' + aux[3]) #
  targets2.append(4)

# [imagem.jpg, 0] -> 0,1: coração normal | 2,3: coração com miocardite | 4: coração doente

info = {"image": images2, "target": targets2}

# Embaralhando o registro

df = pd.DataFrame(info)
df = df.sample(frac = 1)

df.to_csv('data.csv', index = False)


data = pd.read_csv('./data.csv')
# data.target
data.describe

## CNN

In [ ]:
from glob import glob
import os
import PIL
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
from torchvision.models import resnet50, ResNet50_Weights
import torchvision.transforms as T
from torch.utils.data import DataLoader, random_split
from torch import nn, optim
import time
from tqdm.notebook import tqdm

In [ ]:
DATA_DIR = './miocarditeDataset'
NORMAL_DIR = './miocarditeDataset/Normal'
ANORMAL_DIR = './miocarditeDataset/Abnormal'
NORMAL0_DIR = DATA_DIR + "/normal0"
NORMAL1_DIR = DATA_DIR + "/normal1"
ANORMAL2_DIR = DATA_DIR + "/anormal2"
ANORMAL3_DIR = DATA_DIR + "/anormal3"

In [ ]:
data = pd.read_csv('./data.csv')
data.describe

In [ ]:
class CFG:

  epochs =20                              # No. of epochs for training the model
  lr = 0.001                              # Learning rate
  batch_size = 16                         # Batch Size for Dataset

  model_name = 'tf_efficientnet_b4_ns'    # Model name (we are going to import model from timm)
  img_size = 224                          # Resize all the images to be 224 by 224

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("On which device we are on:{}".format(device))

In [ ]:
class MiocarditeDataset(Dataset):

  def __init__(self, csv_file, root_dir, transform = None):
    self.miocardite_frame = pd.read_csv('./data.csv')
    self.root_dir = root_dir
    self.transform = transform

  def __len__(self):
    return len(self.miocardite_frame)

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()

    target = self.miocardite_frame.iloc[idx,1]

    if target == 0:
        dir = NORMAL0_DIR
    elif target == 1:
        dir = NORMAL1_DIR
    elif target == 2:
        dir = ANORMAL2_DIR
    elif target == 3:
        dir = ANORMAL3_DIR
    else:
        dir = SICK_DIR_DS


    img_name = os.path.join(dir, self.miocardite_frame.iloc[idx,0])
    image = Image.open(img_name)

    if self.transform:
      image = self.transform(image)

    sample = (image, target)

    return sample

prep_transform = T.Compose([
                    T.Grayscale(num_output_channels=3),
                    T.ToTensor(),
                    T.Resize(size=(CFG.img_size,CFG.img_size),antialias=None),
                    T.CenterCrop(size=224),
                  ])

miocardite_dataset = MiocarditeDataset(csv_file = './data.csv',
                          root_dir = DATA_DIR,
                          transform = prep_transform)

In [ ]:
img = './410.jpg'
img = Image.open(img)
prep_transform = T.Compose([
                    T.Grayscale(num_output_channels=3),
                    T.Resize(size=(CFG.img_size,CFG.img_size),antialias=None),
                    T.CenterCrop(size=224),
                  ])
img = prep_transform(img)

plt.imshow(img,cmap='gray')
plt.show()


In [ ]:
num_classes = 4

train_ds, valid_ds, test_ds = random_split(miocardite_dataset,[0.7,0.1,0.2])

valid_data_size = len(valid_ds)
train_data_size = len(train_ds)

train_data = DataLoader(train_ds, batch_size = CFG.batch_size, shuffle = True)
valid_data = DataLoader(valid_ds, batch_size = CFG.batch_size, shuffle = True)
test_data = DataLoader(test_ds, batch_size = CFG.batch_size, shuffle = True)

print("Amostras p/ treinamento: ",len(train_ds))
print("Amostras p/ validação: ",len(valid_ds))
print("Amostras p/ teste: ",len(test_ds))

In [ ]:
test_images = []
test_targets = []
for test in test_ds:
    image, target = test
    test_images.append(image)
    test_targets.append(target)

test_csv = {
    "image": test_images,
    "target": test_targets
}

df = pd.DataFrame(test_csv)

In [ ]:
!pip install timm

In [ ]:
from torch import nn
import torch.nn.functional as F
import timm # PyTorch Image Models

model = timm.create_model(CFG.model_name,pretrained=True) #load pretrained model


In [ ]:
#let's update the pretarined model:
for param in model.parameters():
  param.requires_grad=False


#we are updating it as a 2-class classifier:
model.classifier = nn.Sequential(
    nn.Linear(in_features=1792, out_features=625), #1792 is the orginal in_features
    nn.ReLU(), #ReLu to be the activation function
    nn.Dropout(p=0.3),
    nn.Linear(in_features=625, out_features=256),
    nn.ReLU(),
    nn.Linear(in_features=256, out_features=5),
    nn.LogSoftmax(dim=1)
)

In [ ]:
model = model.to(device)

loss_func = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(),lr=CFG.lr)

In [ ]:
def train(model, train_data, loss_func, optimizer):
    # Set to training mode
    model.train()
    # Loss and Accuracy within the epoch
    train_loss = 0.0
    train_acc = 0.0
    train_cumloss = 0.0
    train_cumacc = 0.0

    for i, (inputs, labels) in enumerate(train_data):
        inputs = inputs.to(device)
        labels = labels.to(device)
        # Clean existing gradients
        optimizer.zero_grad()
        # Forward pass - compute outputs on input data using the model
        outputs = model(inputs)
        # Compute loss
        loss = loss_criterion(outputs, labels)
        # Backpropagate the gradients
        loss.backward()
        # Update the parameters
        optimizer.step()
        # Compute the total loss for the batch and add it to train_loss
        train_cumloss += loss.item()
        train_loss += loss.item() * inputs.size(0)
        # Compute the accuracy
        ret, predictions = torch.max(outputs.data, 1)
        correct_counts = predictions.eq(labels.data.view_as(predictions))
        # Convert correct_counts to float and then compute the mean
        acc = torch.mean(correct_counts.type(torch.FloatTensor))
        # Compute total accuracy in the whole batch and add to train_acc
        train_cumacc += acc.item()
        train_acc += acc.item() * inputs.size(0)
#         print("Batch number: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}".format(i, loss.item(), acc.item()))

    return (train_loss, train_acc, train_cumloss, train_cumacc)


In [ ]:
def validade(model, valid_data, loss_func):
    # Set to evaluation mode
    model.eval()
    # Loss and Accuracy within the epoch
    valid_loss = 0.0
    valid_acc = 0.0
    valid_cumloss = 0.0
    valid_cumacc = 0.0

    with torch.no_grad():
        # Validation loop
        for j, (inputs, labels) in enumerate(valid_data):
            inputs = inputs.to(device)
            labels = labels.to(device)
            # Forward pass - compute outputs on input data using the model
            outputs = model(inputs)
            # Compute loss
            loss = loss_criterion(outputs, labels)
            # Compute the total loss for the batch and add it to valid_loss
            valid_cumloss += loss.item()
            valid_loss += loss.item() * inputs.size(0)
            # Calculate validation accuracy
            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))
            # Convert correct_counts to float and then compute the mean
            acc = torch.mean(correct_counts.type(torch.FloatTensor))
            # Compute total accuracy in the whole batch and add to valid_acc
            valid_cumacc += acc.item()
            valid_acc += acc.item() * inputs.size(0)
#             print("Validation Batch number: {:03d}, Validation: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))

    return(valid_loss, valid_acc, valid_cumloss, valid_cumacc)

In [ ]:
epochs = CFG.epochs

loss_criterion = nn.NLLLoss()

conv_train_losses = []
conv_test_losses = []
conv_train_acc = []
conv_test_acc = []

for epoch in range(epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch+1, epochs))
        # Set to training mode
        model.train()

        train_loss, train_acc, train_cumloss, train_cumacc = train(model, train_data, loss_func, optimizer)
        conv_train_losses.append(train_cumloss/len(train_data))
        conv_train_acc.append(train_cumacc/len(train_data))

        valid_loss, valid_acc, valid_cumloss, valid_cumacc = validade(model, valid_data, loss_func)
        conv_test_losses.append(valid_cumloss/len(valid_data))
        conv_test_acc.append(valid_cumacc/len(valid_data))

avg_train_loss = train_loss/train_data_size
avg_train_acc = train_acc/float(train_data_size)
# Find average training loss and training accuracy
avg_valid_loss = valid_loss/valid_data_size
avg_valid_acc = valid_acc/float(valid_data_size)
history = ([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])
epoch_end = time.time()
print("Epoch : {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, nttValidation : Loss : {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(epoch, avg_train_loss, avg_train_acc*100, avg_valid_loss, avg_valid_acc*100, epoch_end-epoch_start))

## Carregando o modelo

In [ ]:
def plot_losses(losses):
    fig = plt.figure(figsize=(13, 5))
    ax = fig.gca()
    for loss_name, loss_values in losses.items():
        ax.plot(loss_values, label=loss_name)
    ax.legend(fontsize="16")
    ax.set_xlabel("Iteration", fontsize="16")
    ax.set_ylabel("Loss", fontsize="16")
    ax.set_title("Loss vs iterations", fontsize="16")
    plt.savefig('./graficos/70-10-20_20eph_0001LR_Loss_ART.png', format='png')

def plot_acc(acc):
    fig = plt.figure(figsize=(13, 5))
    ax = fig.gca()
    for acc_name, acc_values in acc.items():
        ax.plot(acc_values, label=acc_name)
    ax.legend(fontsize="16")
    ax.set_xlabel("Epochs", fontsize="16")
    ax.set_ylabel("Accuracy", fontsize="16")
    ax.set_title("Accuracy vs iterations", fontsize="16")
    plt.savefig('./graficos/70-10-20_20eph_0001LR_Acc_ART.png', format='png')


In [ ]:
conv_losses = {"Train Loss": conv_train_losses, "Test Loss": conv_test_losses}
plot_losses(conv_losses)

In [ ]:
conv_acc = {"Train Acc": conv_train_acc, "Test Acc": conv_test_acc}
plot_acc(conv_acc)

In [ ]:
def predict(model, teste):
    img, target = teste
    test_image = transf(img)
#     plt.imshow(test_image)
    test_image_tensor = img
    if torch.cuda.is_available():
        test_image_tensor = test_image_tensor.view(1, 3, 224, 224).cuda()
    else:
        test_image_tensor = test_image_tensor.view(1, 3, 224, 224)
    with torch.no_grad():
        model.eval()
        # Model outputs log probabilities
        out = model(test_image_tensor)
        ps = torch.exp(out)
#         topk, topclass = ps.topk(1, dim=1)
        class_prob = torch.softmax(out, dim=1)
        # get most probable class and its probability:
        class_prob, topclass1 = torch.max(class_prob, dim=1)
        print("Out: ",out)
        print("Predict: ",topclass1)
        print("Prob: ",class_prob)
        print("Target: ",target)
        print("======================\n")



In [ ]:
# Salvando modelo
torch.save(model.state_dict(), 'model/efficientnet_model_weights_70_10_20_0001_ART.pth')

In [ ]:
def make_confusion_matrix(model, loader, n_classes):
  confusion_matrix = torch.zeros(n_classes, n_classes, dtype=torch.int64)
  with torch.no_grad():
    for i, (imgs, target) in enumerate(loader):
      imgs = imgs.to(device)
      target = target.to(device)
      outputs = model(imgs)
      _, predicted = torch.max(outputs, 1)
      for t, p in zip(torch.as_tensor(target, dtype=torch.int64).view(-1),
                      torch.as_tensor(predicted, dtype=torch.int64).view(-1)):
        confusion_matrix[t, p] += 1
  return confusion_matrix

def evaluate_accuracy(model, dataloader, classes, verbose=True):
  # prepare to count predictions for each class
  correct_pred = {classname: 0 for classname in classes}
  total_pred = {classname: 0 for classname in classes}

  confusion_matrix = make_confusion_matrix(model, dataloader, len(classes))
  if verbose:
    total_correct = 0.0
    total_prediction = 0.0
    for i, classname in enumerate(classes):
      correct_count = confusion_matrix[i][i].item()
      class_pred = torch.sum(confusion_matrix[i]).item()

      total_correct += correct_count
      total_prediction += class_pred

      accuracy = 100 * float(correct_count) / class_pred
      print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                    accuracy))
  print("Global acccuracy is {:.1f}".format(100 * total_correct/total_prediction))
  return confusion_matrix


In [ ]:
CATEGORIES = ['normal0','normal1','anormal2','anormal3','sick']

conv_confusion_matrix = evaluate_accuracy(model, valid_data, CATEGORIES)

In [ ]:
def show_CMR(image):
    transf = T.ToPILImage()
    image = transf(image)
    plt.imshow(image,cmap='gray')
    plt.show()

## Testes

In [ ]:
!pip install captum
!pip uninstall matplotlib -y
!pip install matplotlib==3.5

In [ ]:
from captum.attr import IntegratedGradients
from captum.attr import GradientShap
from captum.attr import Occlusion
from captum.attr import NoiseTunnel
from captum.attr import visualization as viz


In [ ]:
conv_confusion_matrix = evaluate_accuracy(model, test_data, CATEGORIES)

In [ ]:
image_test, target_test = test_ds[0]

print(image_test.shape)
if torch.cuda.is_available():
    input = image_test.view(1, 3, 224, 224).cuda()
else:
    input = image_test.view(1, 3, 224, 224)

output = model(input)
output = F.softmax(output, dim=1)
prediction_score, pred_label_idx = torch.topk(output, 1)

print('Predicted:', pred_label_idx, '(', prediction_score.squeeze().item(), ')')
print('Target: ',target_test)

occlusion = Occlusion(model)

attributions_occ = occlusion.attribute(input,
                                       strides = (3, 8, 8),
                                       target=pred_label_idx,
                                       sliding_window_shapes=(3,15, 15),
                                       baselines=(1, 1, 1))

figure, ax = viz.visualize_image_attr_multiple(np.transpose(attributions_occ.squeeze().cpu().detach().numpy(), (1,2,0)),
                                      np.transpose(image_test.squeeze().cpu().detach().numpy(), (1,2,0)),
                                      ["blended_heat_map"],
                                      ["all"],
                                      show_colorbar=True,
                                      outlier_perc=2,
                                      fig_size=(3, 3)
                                     )


In [ ]:
TESTS_OCC_DIR = './testesOcclusion/*'
pathImgTest = glob(os.path.join(TESTS_OCC_DIR,'*.jpg'))

prep_transform = T.Compose([
                    T.Grayscale(num_output_channels=3),
                    T.ToTensor(),
                    T.Resize(size=(CFG.img_size,CFG.img_size),antialias=None),
                    T.CenterCrop(size=224),
                  ])

for img in pathImgTest:
    image = Image.open(img)
    image = prep_transform(image)
    img_tensor = image


    if torch.cuda.is_available():
        image = image.view(1, 3, 224, 224).cuda()
    else:
        image = image.view(1, 3, 224, 224)
    with torch.no_grad():
        model.eval()
        out = model(image)
        ps = torch.exp(out)
        class_prob = torch.softmax(out, dim=1)
        class_prob, topclass1 = torch.max(class_prob, dim=1)

        print("Imagem: ",img)
        print("Out: ",out)
        print("Predict: ",topclass1)
        print("Prob: ",class_prob)
        print("======================\n")

        occlusion = Occlusion(model)

        attributions_occ = occlusion.attribute(input,
                                               strides = (3, 8, 8),
                                               target=topclass1,
                                               sliding_window_shapes=(3,15, 15),
                                               baselines=(0,0,0))

        _ = viz.visualize_image_attr_multiple(np.transpose(attributions_occ.squeeze().cpu().detach().numpy(), (1,2,0)),
                                              np.transpose(img_tensor.squeeze().cpu().detach().numpy(), (1,2,0)),
                                              ["blended_heat_map"],
                                              ["all"],
                                              show_colorbar=True,
                                              outlier_perc=2,
                                             )


In [ ]:
err = 0

transform_normalize = T.Normalize(
     mean=[0.485, 0.456, 0.406],
     std=[0.229, 0.224, 0.225]
 )


for test in test_ds:
    img, target = test
    image = img
    if torch.cuda.is_available():
        image = img.view(1, 3, 224, 224).cuda()
    else:
        image = img.view(1, 3, 224, 224)

    input = transform_normalize(image)
    with torch.no_grad():
        model.eval()
        # Model outputs log probabilities
        out = model(image)




        ps = torch.exp(out)
    #         topk, topclass = ps.topk(1, dim=1)
        class_prob = torch.softmax(out, dim=1)
        # get most probable class and its probability:
        class_prob, topclass1 = torch.max(class_prob, dim=1)
        if topclass1 != target:
            print("ERRO")
            err+=1

        print("Out: ",out)
        print("Predict: ",topclass1)
        print("Target: ", target)
        print("Prob: ",class_prob)
        print("======================\n")

        occlusion = Occlusion(model)

        attributions_occ = occlusion.attribute(input,
                                               strides = (3, 8, 8),
                                               target=topclass1,
                                               sliding_window_shapes=(3,15, 15),
                                               baselines=(0,0,0))

        _ = viz.visualize_image_attr_multiple(np.transpose(attributions_occ.squeeze().cpu().detach().numpy(), (1,2,0)),
                                              np.transpose(img.squeeze().cpu().detach().numpy(), (1,2,0)),
                                              ["blended_heat_map"],
                                              ["all"],
                                              show_colorbar=True,
                                              outlier_perc=2,
                                             )

print("Result: ",1 - err/len(test_ds))
print("Quantidade de amostras analisadas: ",len(test_ds))
print("Erros: ",err)
